In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from scipy.stats import uniform
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#### Importamos librerías

In [ ]:
df = pd.read_csv("../../data/preprocessed/base.csv")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
df.shape

#### Generación de Dataset

In [ ]:
# Determinar vida útil de cada producto (primer y último periodo)
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()
periodos_producto = []
for _, row in vida_producto.iterrows():
    periodos = pd.date_range(start=row["min"], end=row["max"], freq="MS")
    for p in periodos:
        periodos_producto.append((p, row["product_id"]))
df_producto_periodo = pd.DataFrame(periodos_producto, columns=["periodo_dt", "product_id"])



# Agregar columna periodo en formato AAAAMM
df_producto_periodo["periodo"] = df_producto_periodo["periodo_dt"].dt.strftime("%Y%m").astype(int)
df_producto_periodo.drop(columns=['periodo_dt'],inplace=True)
df_producto_periodo = df_producto_periodo.sort_values(by=["product_id", "periodo"], ascending=True).reset_index(drop=True)



# Sumarizacion de toneladas
toneladas_vendidas = df.copy()
toneladas_vendidas = df.groupby(['periodo', 'product_id']).agg({
    'tn': 'sum',
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum'
}).reset_index()


# Unir con las toneladas efectivamente vendidas (tn)
df_merge = df_producto_periodo.merge(toneladas_vendidas[["periodo", "product_id", "tn","cust_request_qty", "cust_request_tn"]],
                             on=["periodo", "product_id"],
                             how="left")
df_merge["tn"] = df_merge["tn"].fillna(0)


# Unir con productos y stocks
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
df_merge = df_merge.merge(productos[['product_id', 'cat1', 'cat2', 'cat3','brand','sku_size']], on='product_id', how='left')
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
df_merge = df_merge.merge(stocks[['product_id', 'periodo', 'stock_final']], on=['product_id', 'periodo'], how='left')
print(df_merge.shape)


# precios cuidados
df_precios = df[['product_id', 'periodo_dt', 'plan_precios_cuidados']].drop_duplicates()
periodos_producto = []
for _, row in vida_producto.iterrows():
    product_id = row['product_id']
    min_fecha = row['min']
    max_fecha = row['max']
    periodos = pd.date_range(start=min_fecha, end=max_fecha, freq='MS')
    for p in periodos:
        periodos_producto.append((product_id, p))

df_periodos = pd.DataFrame(periodos_producto, columns=['product_id', 'periodo_dt'])
df_final = df_periodos.merge(df_precios, on=['product_id', 'periodo_dt'], how='left')
df_final["periodo"] = df_final["periodo_dt"].dt.strftime("%Y%m").astype(int)
df_final.drop(columns=['periodo_dt'],inplace=True)

df_merge = df_merge.merge(df_final[['product_id', 'periodo', 'plan_precios_cuidados']], on=['product_id', 'periodo'], how='left')
df_merge.shape

#### Feature Engineering

In [ ]:
ts = df_merge.copy()
ts['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')

Categóricas

In [ ]:
# Convertir las columnas de categoría a tipo 'category' para que las detecte LGBM
ts['cat1'] = ts['cat1'].astype('category')
ts['cat2'] = ts['cat2'].astype('category')
ts['cat3'] = ts['cat3'].astype('category')
ts['brand'] = ts['brand'].astype('category')
ts['sku_size'] = ts['sku_size'].astype('category')

Desagregación de Fechas

In [ ]:
# Crear características adicionales
ts['crisis'] = (ts['periodo_dt'].dt.year == 2019) & (ts['periodo_dt'].dt.month == 8)
ts['quarter'] = ts['periodo_dt'].dt.quarter
ts['month'] = ts['periodo_dt'].dt.month
ts['year'] = ts['periodo_dt'].dt.year
ts['season'] = ts['periodo_dt'].apply(lambda x: 1 if x.month in [6, 7, 8] else 0)
ts['tn_diff'] = ts.groupby('product_id')['tn'].diff()
ts['rolling_mean'] = ts.groupby('product_id')['tn'].rolling(window=3).mean().reset_index(level=0, drop=True)
ts['interaction'] = ts['year'] * ts['month']

Normalización

In [ ]:
# Normalización por producto
# ts['tn_norm'] = ts.groupby('product_id')['tn'].transform(lambda x: (x - x.mean()) / x.std())

Lags en Toneladas

In [ ]:
# Agregar lags a las toneladas
for lag in range(0, 13):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

Fecha Primera y Ultima Venta

In [ ]:
# Identificar el primer y último periodo de ventas para cada producto
ts['first_sale'] = ts.groupby('product_id')['periodo_dt'].transform('min')
ts['last_sale'] = ts.groupby('product_id')['periodo_dt'].transform('max')
ts['months_since_launch'] = (ts['periodo_dt'] - ts['first_sale']).dt.days // 30  # en meses
# Calculamos el tiempo desde la primera venta para cada registro

Grado de Madurez del Producto

In [ ]:
# Crear una categoría de madurez basada en el tiempo desde la primera venta
conditions = [
    (ts['months_since_launch'] < 6),
    (ts['months_since_launch'] >= 6) & (ts['months_since_launch'] < 18),
    (ts['months_since_launch'] >= 18) & (ts['months_since_launch'] < 30),
    (ts['months_since_launch'] >= 30)
]
choices = ['new', 'growth', 'mature', 'decline']
ts['grado_de_madurez'] = np.select(conditions, choices, default='unknown')

# One-Hot Encode the grado_de_madurez feature
ts = pd.get_dummies(ts, columns=['grado_de_madurez'], drop_first=True)

Pesos

In [ ]:
# Paso 1: Calcular la suma total por producto
participacion = ts.groupby('product_id')['tn'].sum()
# Paso 2: Calcular el total global
total_global = participacion.sum()
# Paso 3: Calcular la proporción por producto
participacion = participacion / total_global
participacion.name = 'participacion_tn'
# Paso 4: Merge con el DataFrame original
ts = ts.merge(participacion, on='product_id', how='left')

Stock ratio

In [ ]:
# stock de prodctos: velocidad de rotacion
ts['stock_ratio'] = ts['tn'] / ts['stock_final']
ts['stock_ratio'] = ts.apply(
    lambda x: x['tn'] / x['stock_final'] if x['stock_final'] > 0 else 0,
    axis=1
)
ts['stock_ratio'] = ts['stock_ratio'].replace([np.inf, -np.inf], 0).fillna(0)

Crecimiento de stock

In [ ]:
# Crecimiento del stock entre periodos:  Útil para detectar si el producto está acumulando inventario o escaseando.
ts['stock_growth'] = ts.groupby('product_id')['stock_final'].pct_change() #  calcula el cambio porcentual entre el valor actual y el valor anterior en la columna stock_final para cada grupo. 
ts['stock_growth'] = ts['stock_growth'].replace([np.inf, -np.inf], 0).fillna(0)

Stock vs Promedio

In [ ]:
# Relación stock actual vs. promedio histórico
avg_stock = ts.groupby('product_id')['stock_final'].transform('mean')
ts['stock_vs_avg'] = ts['stock_final'] / avg_stock
ts['stock_vs_avg'] = ts['stock_vs_avg'].replace([np.inf, -np.inf], 0).fillna(0)

Lags de stock

In [ ]:
# Stock lagueado: Ideal para que LightGBM aprenda con información de meses previos.
ts['stock_lag1'] = ts.groupby('product_id')['stock_final'].shift(1)
ts['stock_lag1'] = ts['stock_lag1'].replace([np.inf, -np.inf], 0).fillna(0)
ts['stock_lag2'] = ts.groupby('product_id')['stock_final'].shift(2)
ts['stock_lag2'] = ts['stock_lag2'].replace([np.inf, -np.inf], 0).fillna(0)

Categorizar el nivel de stock

In [ ]:

ts['stock_level'] = pd.qcut(ts['stock_final'], q=4, labels=['Muy bajo', 'Bajo', 'Medio', 'Alto'])
ts['stock_level'] = ts['stock_level'].cat.codes

Porcentaje vendido al top 13 de clientes

In [ ]:
# Porcentaje vendido al top 13 de clientes: Nueva columna que, para cada combinación periodo-producto, indique qué porcentaje de toneladas fue vendido a los top 13 clientes.
# Paso 1: Identificar los top 13 clientes
df_copy = df.copy() 
top_13 = (df_copy.groupby('customer_id')['tn'].sum()
          .sort_values(ascending=False)
          .head(13)
          .index)
# Paso 2: Calcular toneladas por periodo-producto para top13
df_copy['is_top13'] = df_copy['customer_id'].isin(top_13)
agregado_total = df_copy.groupby(['periodo', 'product_id'])['tn'].sum()
agregado_top13 = df_copy[df_copy['is_top13']].groupby(['periodo', 'product_id'])['tn'].sum()
# Paso 3: Crear DataFrame de proporción
df_prop = (agregado_top13 / agregado_total).reset_index(name='porcentaje_top13')
# Paso 4: Merge con tu dataset original
ts = ts.merge(df_prop, on=['periodo', 'product_id'], how='left')

Numero de clientes distintos por producto y mes

In [ ]:
# Numero de clientes distintos por producto y mes: Esto indica cuán diversificada es la demanda por producto en cada periodo.
clientes_distintos = df.groupby(['periodo', 'product_id'])['customer_id'].nunique().reset_index(name='n_clientes')
ts = ts.merge(clientes_distintos, on=['periodo', 'product_id'], how='left')

Indice de Herfindahl

In [ ]:

# Concentración (índice de Herfindahl): Podés calcular el índice de concentración por producto y mes. 
# El índice de Herfindahl es la suma de los cuadrados de las participaciones de los clientes:
# Calcular la participación de cada cliente por periodo-producto
participaciones = df.groupby(['periodo', 'product_id', 'customer_id'])['tn'].sum()
# Calcular el índice de Herfindahl-Hirschman
participaciones_pct = participaciones.groupby(['periodo', 'product_id']).apply(
    lambda x: ((x / x.sum())**2).sum()
).reset_index(name='hh_index')
# Merge con tu dataframe agregado por periodo y producto
ts = ts.merge(participaciones_pct, on=['periodo', 'product_id'], how='left')

Tasa de repetición de clientes

In [ ]:
# % de clientes que ya compraron el producto en el período anterior
# Paso 1: Agrupar y obtener clientes únicos por período y producto
clientes_por_periodo = df.groupby(['periodo', 'product_id'])['customer_id'].unique().reset_index()
# Paso 2: Ordenar
clientes_por_periodo = clientes_por_periodo.sort_values(['product_id', 'periodo'])
# Paso 3: Shift preservando la estructura (usando transform)
clientes_por_periodo['clientes_prev'] = (
    clientes_por_periodo.groupby('product_id')['customer_id']
    .transform(lambda x: x.shift(1))
)
# Paso 4: Función corregida para tasa de repetición
def tasa_repeticion(row):
    clientes_actuales = set(row['customer_id']) if isinstance(row['customer_id'], np.ndarray) else set()
    clientes_anteriores = set(row['clientes_prev']) if isinstance(row['clientes_prev'], np.ndarray) else set()
    
    if not clientes_actuales:
        return 0.0
    
    repetidos = clientes_actuales & clientes_anteriores
    return len(repetidos) / len(clientes_actuales)
clientes_por_periodo['tasa_repeticion'] = clientes_por_periodo.apply(tasa_repeticion, axis=1)
# Resultado
resultado = clientes_por_periodo[['periodo', 'product_id', 'tasa_repeticion']]
ts = ts.merge(resultado, on=['periodo', 'product_id'], how='left')


Clientes nuevos para ese producto

In [ ]:
# Construir un historial de compras por cliente-producto
df_copy = df.copy()
df_copy['first_purchase'] = df_copy.groupby(['customer_id', 'product_id'])['periodo'].transform('min')
# Cliente nuevo = primera compra de ese producto en ese mes
df_copy['cliente_nuevo'] = (df_copy['periodo'] == df_copy['first_purchase']).astype(int)
# Agregar a nivel periodo-producto
clientes_nuevos = df_copy.groupby(['periodo', 'product_id'])['cliente_nuevo'].sum().reset_index()
# Merge con el DataFrame original
ts = ts.merge(clientes_nuevos, on=['periodo', 'product_id'], how='left')

Se podría agregar esta segmentación de cliente...

In [ ]:
# conteo_customers = (
#     df.groupby('customer_id')
#     .size()
#     .reset_index(name='cantidad_compras')
# )

# conteo_customers['media'] = conteo_customers['cantidad_compras'].median()
# conteo_customers['ds'] = conteo_customers['cantidad_compras'].std()
# conteo_customers['q10'] = conteo_customers['cantidad_compras'].quantile(0.1)
# conteo_customers['q90'] = conteo_customers['cantidad_compras'].quantile(0.9)

# def clasificar_frecuencia(frecuencia,median,q10,q90):

#     if frecuencia <= q10:
#         return "🕸 Inactivo"
#     elif  frecuencia <= median :
#         return "🟡 Ocasional"
#     elif  frecuencia <= q90:
#         return "🟢 Frecuente"
#     else:
#         return "🔵 Fiel"
    

# conteo_customers['segmento_frecuencia'] = conteo_customers.apply(
#     lambda row: clasificar_frecuencia(
#         row['cantidad_compras'],
#         row['media'],  # <- Esto no tiene sentido en tu caso, porque 'media' es igual para todos
#         row['q10'],
#         row['q90']
#     ),
#     axis=1
# )



# # Supongamos que df tiene una columna 'segmento' asociada al cliente
# segmento_dominante = (
#     df.groupby(['periodo', 'product_id', 'segmento'])['tn']
#     .sum()
#     .reset_index()
#     .sort_values(['periodo', 'product_id', 'tn'], ascending=False)
#     .drop_duplicates(subset=['periodo', 'product_id'])
#     .rename(columns={'segmento': 'segmento_dominante'})
# )

Promedio histórico de toneladas por cliente (en un mes): promedio de compra

In [ ]:
promedio_tn_cliente = (
    df.groupby(['product_id', 'customer_id'])['tn']
    .mean()
    .reset_index()
    .groupby('product_id')['tn']
    .mean()
    .reset_index()
    .rename(columns={'tn': 'prom_tn_cliente'})
)
ts = ts.merge(promedio_tn_cliente, on='product_id', how='left')

Varianza de las toneladas por cliente (en un mes)

In [ ]:
# Dispersión en tamaño de compra
var_tn_cliente = (
    df.groupby(['periodo', 'product_id'])['tn']
    .std()
    .reset_index()
    .rename(columns={'tn': 'std_tn_cliente'})
)
ts = ts.merge(var_tn_cliente, on=['periodo', 'product_id'], how='left')

Coeficiente de Gini de participación de clientes

In [ ]:
# Coeficiente de Gini de participación de clientes: desigualdad en la distribución de compras
def gini(array):
    array = np.sort(np.array(array))
    n = len(array)
    if n == 0:
        return np.nan
    cumx = np.cumsum(array, dtype=float)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

gini_por_producto = (
    df.groupby(['periodo', 'product_id'])['tn']
    .apply(gini)
    .reset_index()
    .rename(columns={'tn': 'gini_clientes'})
)
ts = ts.merge(gini_por_producto, on=['periodo', 'product_id'], how='left')

Fechas

In [ ]:
ts['periodo_dt'] = ts['periodo_dt'].dt.year * 100 + ts['periodo_dt'].dt.month
ts['first_sale'] = ts['first_sale'].dt.year * 100 + ts['first_sale'].dt.month
ts['last_sale'] = ts['last_sale'].dt.year * 100 + ts['last_sale'].dt.month

Target

In [ ]:
ts['tn_target'] = ts['tn'].shift(-2)

#### Función para calcular pesos

In [ ]:
def calcular_pesos(ts):
    ventas_totales = ts.groupby('product_id')['tn'].sum()
    pesos = ventas_totales / ventas_totales.sum()
    return pesos

#### Entrenamiento de Ensemble: LGB + RF + XGB

In [ ]:
# quito 201911 y 201912
ts = ts[:-2]

X = ts.drop(columns=['tn_target'])
y = ts['tn_target']

pesos_ventas = calcular_pesos(ts)

# Validación temporal en lugar de train_test_split
tscv = TimeSeriesSplit(n_splits=5)
X_train, X_test, y_train, y_test = None, None, None, None
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()  # Hacer una copia explícita
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Codificar las características categóricas 'cat1', 'cat2', 'cat3'
for col in ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']:
    X_train.loc[:, col] = X_train[col].astype('category').cat.codes
    X_test.loc[:, col] = X_test[col].astype('category').cat.codes

# Obtener los pesos para el conjunto de entrenamiento
pesos_entrenamiento = pesos_ventas.loc[X_train['product_id']].values

# Definir el espacio de búsqueda de hiperparámetros para LightGBM
param_dist = {
    'num_leaves': [15, 31, 50, 70, 128],  # [31, 50, 70, 128],
    'max_depth': [-1, 10, 20, 30],  # [-1, 10, 20, 30],
    'learning_rate': uniform(0.01, 0.1),
    'n_estimators': [100, 200, 500, 700],  # [100, 200, 500],
    'min_child_samples': [10, 20, 30],  # [20, 30, 40],
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2),
    'reg_alpha': uniform(0.0, 0.5),
    'reg_lambda': uniform(0.0, 0.5)
}

# Definir el modelo de LightGBM con RandomizedSearchCV
lgb_model = lgb.LGBMRegressor(random_state=42)
random_search = RandomizedSearchCV(lgb_model, param_distributions=param_dist, n_iter=100, cv=5, verbose=1, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train, sample_weight=pesos_entrenamiento)

print(f"Best parameters found: {random_search.best_params_}")

# Crear y ajustar el modelo de Random Forest con pesos
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train, sample_weight=pesos_entrenamiento)

# Crear y ajustar el modelo de XGBoost con pesos
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train, sample_weight=pesos_entrenamiento)

# Obtener el mejor modelo de LightGBM
lgb_model = random_search.best_estimator_

# Crear el modelo de ensemble con VotingRegressor
ensemble_model = VotingRegressor(estimators=[
    ('lgb', lgb_model),
    ('rf', rf_model),
    ('xgb', xgb_model)
])

# Ajustar el modelo de ensemble
ensemble_model.fit(X_train, y_train, sample_weight=pesos_entrenamiento)

# Predecir en el conjunto de prueba
y_pred = ensemble_model.predict(X_test)

# Calcular métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Ensemble Model MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

joblib.dump(ensemble_model, './models/model_E1.pkl')

#### Lista de Productos a Predecir

In [ ]:
# Levantamos
productos_a_predecir = pd.read_csv("../../data/raw/product_id_apredecir201912.csv")
productos_filtrados = productos_a_predecir['product_id'].unique()
filtro = (ts['periodo'] == 201912) & (ts['product_id'].isin(productos_filtrados))

# Subconjunto filtrado
ts_filtrado = ts[filtro]
for col in ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']:
    ts_filtrado.loc[:, col] = ts_filtrado[col].astype('category').cat.codes
    
# Quitamos target
ts_filtrado.drop(columns=['tn_target'], inplace=True)

#### Predecimos y Guardamos los Resultados

In [ ]:
# Predecimos
y_pred = ensemble_model.predict(ts_filtrado)

# Calculamos los pesos
resultados = ts_filtrado[['product_id', 'periodo']].copy()
resultados['tn_pred'] = y_pred
pesos_ventas = calcular_pesos(ts)

for idx, row in resultados.iterrows():
    peso = pesos_ventas.get(row['product_id'],0)
    # Guardar el nuevo valor
    resultados.at[idx, 'tn_pred'] = row['tn_pred'] * peso

# Mostrar resultados
print(resultados)
resultados.drop(columns=['periodo'],inplace=True)
resultados.rename(columns={'tn_pred':'tn'}, inplace=True)
resultados.to_csv("../../output/03-modelo-ensamblador.csv", index=False, sep=',')